In [1]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

root_dir = './09.resource/dogcat_data/'
target_train_path = root_dir + 'train/'
target_val_path = root_dir + 'val/'

img_height, img_width = 100, 100
nb_train_samples = 1
nb_validation_samples = 1
epochs = 1
batch_size = 1

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
)

val_datagen = ImageDataGenerator(
    rescale=1.0/255,
)

train_generator = train_datagen.flow_from_directory(
    target_train_path,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical',
)


validation_generator = val_datagen.flow_from_directory(
    target_val_path,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical',
)


checkpoint_path = './09.resource/model.h5'
checkpoint = keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor='val_acc',
    verbose=2,
    save_weights_only=False,
    save_best_only=True,
    mode='max',
)

base_model = keras.applications.VGG16(
    weights='imagenet',
    include_top=False,
)

x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation=keras.activations.relu)(x)
x = keras.layers.Dropout(0.5)(x)

predictions = keras.layers.Dense(2, activation='softmax')(x)



model = keras.Model(
    inputs=base_model.input,
    outputs=predictions,
)

for layer in base_model.layers:
    layer.trainable = True


model.compile(
    optimizer=keras.optimizers.Adam(lr=0.0001, decay=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


/home/haormj/Develop/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 78 images belonging to 2 classes.
Found 24 images belonging to 2 classes.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples,
    callbacks=[checkpoint],
    verbose=1,
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
    9/10000 [..............................] - ETA: 48:54:33 - loss: 0.7093 - acc: 0.5719